In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.schema.runnable import RunnablePassthrough

from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = DirectoryLoader(
    path='../',
    glob="**/files/*.txt"
)

docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

chain.invoke("What is benefits of using LCEL")



AIMessage(content='The benefits of using LCEL (LangChain Expression Language) include:\n\n1. Unified Interface: Every LCEL object implements the Runnable interface, which allows chains of LCEL objects to support common invocation methods. This makes it easy to build complex chains from basic components.\n\n2. Composition Primitives: LCEL provides primitives that simplify the composition of chains, allowing for parallelization, fallbacks, dynamic configuration, and more.\n\n3. Streaming Support: LCEL chains are designed to have the best possible time-to-first-token, allowing for streaming of output in incremental chunks.\n\n4. Async Support: LCEL chains can be called both synchronously and asynchronously, enabling the same code to be used for prototypes and production with great performance and the ability to handle concurrent requests.\n\n5. Optimized Parallel Execution: LCEL automatically executes steps in parallel when possible, reducing latency.\n\n6. Retries and Fallbacks: LCEL cha